In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# Get all pages into a page scrapping list
# Define base url. Note that main page list as shown in landing site is not updated with full directory. At the bottom,
# navigate to page 2 and back to page 1. New page 1 list is not the same as Landing Site page listing.
# Thus, method to obtain all updated comprehensive list is to apply a sorting filter ("Newest First") and start with all pages listed.
landing_page = "https://ngo.mycitizen.net/"
page_list=[]

# Obtain page request
landing_page_request = requests.get(landing_page)
# Parse page request into soup
landing_page_soup=BeautifulSoup(landing_page_request.text,"html.parser")

# Obtain sorted page filter url
sorted_page_anchor = landing_page_soup.find("a", attrs={"data-container":".sabai-directory-listings-container"})
sorted_page_url = sorted_page_anchor['href']
print(sorted_page_url)

https://ngo.mycitizen.net/directory?sort=newest&category=0&zoom=15&is_mile=0&directory_radius=0&view=list


In [3]:
# Sorted page listings to obtain all pages of listing
sort_page_list=[]

# sorted_page_soup = BeautifulSoup(requests.get(sorted_page_url).text,"html.parser")
# To find all the urls found in the navigation buttons
# for i in sorted_page_soup.find_all("a",class_="sabai-btn sabai-btn-default sabai-btn-sm"):
#     print(i['href'])
# Output print of urls obtained
# https://ngo.mycitizen.net/directory?p=2&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest
# https://ngo.mycitizen.net/directory?p=2&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest
# https://ngo.mycitizen.net/directory?p=2&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest
       
# Note that the website is not updated correctly, as the navigation links at the bottom are all linked to "p=2" in the url.
# Thus manually key in page number to get compiled filtered pages to scrape.
for i in range(3):
    p = i+1
    sort_page_list.append("https://ngo.mycitizen.net/directory?p={}&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest".format(p))

print(sort_page_list)

['https://ngo.mycitizen.net/directory?p=1&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest', 'https://ngo.mycitizen.net/directory?p=2&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest', 'https://ngo.mycitizen.net/directory?p=3&category=0&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest']


In [4]:
org_list= []
listing_url_list = []

for sort_page in sort_page_list:
    
    sort_page_url = requests.get(sort_page)
    
    sort_page_soup = BeautifulSoup(sort_page_url.text,"html.parser")
    
    for soup in sort_page_soup.find_all('div', class_='sabai-directory-title'):
        # Get Organisation Title
        org_list.append(soup.find('a',href=True)['title'])
        # Get Organisation listing page
        listing_url_list.append(soup.find('a',href=True)['href'])

In [5]:
# Prepare all the required lists (Category, Address,)
category_list=[]
address_list = []
telephone_list=[]
mobile_list=[]
email_list = []
website_list = []
twitter_list = []
facebook_list =[]
description_list =[]
local_name_list = []
activities_list = []
regid_list = []
yof_list = []
further_web_list = []

for url in listing_url_list:
    url_soup = BeautifulSoup(requests.get(url).text,"html.parser")

    container = url_soup.find("div",{"id":"sabai-body"})
    
    # Category class of organisation
    cat_class = url_soup.find('div', class_="sabai-directory-category").find_all("a",href=True)

    if cat_class is None:
        category_list.append("NA")
    else:
        #cat_class_list[0].text to see first category. Note that each word has 1 white space in front.
        # concatenate all the categories into single string, but separated by "/"
        text = "/".join(i.text[1:] for i in cat_class) # eliminate initial white space
        category_list.append(text)

    # Address location information
    loc_add = url_soup.find('span',class_ = "sabai-googlemaps-address")
    if loc_add is None:
        address_list.append("NA")
    else:
        address_list.append(loc_add.text[1:])

    # find contact information
    # Telephone
    contact_telephone = container.find("div", class_="sabai-directory-contact-tel")
    if contact_telephone is None:
        telephone_list.append("NA")
    else:
        telephone_list.append(contact_telephone.find("a",href=True).text)

    # Mobile
    contact_mobile = container.find("div",class_="sabai-directory-contact-mobile")
    if contact_mobile is None:
        mobile_list.append("NA")
    else:
        mobile_list.append(contact_mobile.find("a",href=True).text)

    # Email
    contact_email = container.find("div", class_= "sabai-directory-contact-email")
    if contact_email is None:
        email_list.append("NA")
    else:
        email_list.append(contact_email.find("a",href=True).text)

    # Website
    contact_website = container.find("div",class_="sabai-directory-contact-website")
    if contact_website is None:
        website_list.append("NA")
    else:
        website_list.append(contact_website.find("a", href=True).text)

    # Twitter
    contact_twitter = container.find("a",class_="sabai-directory-social-twitter")
    if contact_twitter is None:
        twitter_list.append("NA")
    else:
        twitter_list.append(contact_twitter['href'])

    # Facebook
    contact_facebook = container.find("a",class_="sabai-directory-social-facebook")
    if contact_facebook is None:
        facebook_list.append("NA")
    else:
        facebook_list.append(contact_facebook['href'])

    # Description body
    body_text = container.find("div","sabai-directory-body")

    if body_text is None:
        description_list.append("NA")
    else:
        description_list.append(body_text.text)

    # name in local lang
    local_name = container.find("div","sabai-field-name-field-title-own")
    if local_name is None:
        local_name_list.append("NA")
    else:
        local_name_list.append(local_name.find("div","sabai-field-value").text)

    # activities
    activities= container.find("div","sabai-field-name-field-activities")
    if activities is None:
        activities_list.append("NA")
    else:
        activities_list.append(activities.text)

    # registration id
    regid = container.find("div","sabai-field-name-field-reg-id")
    if regid is None:
        regid_list.append("NA")
    else:
        regid_list.append(regid.find("div","sabai-field-value").text)

    # year of foundation
    yof = container.find("div","sabai-field-name-field-founded")
    if yof is None:
        yof_list.append("NA")
    else:
        # Remove comma and convert to int
        yof_list.append(int(yof.find("div","sabai-field-value").text.replace(",","")))

    # Further websites
    further_web = container.find("div","sabai-field-name-field-websites")
    #.find_all("a",href=True)
    if further_web is None:
        further_web_list.append("NA")
    else:
        # append all further websites links separated by " | "
        further_web_list.append(" | ".join(i['href'] for i in further_web.find_all("a",href=True)))

In [6]:
# Create Combined Contact List
contact_list=[]

for i in range(len(org_list)):
    contact_list.append("Tel: {}, Mob: {}".format(telephone_list[i],mobile_list[i]))
    

In [7]:
# Create Country list
country_list =[]

for i in address_list:
    
    # Myanmar
    if any(x in i.lower() for x in ["yangon","chin state","moulmein","ayeyarwady","bogale","mandalay","myanmar"]):
        country_list.append("Myanmar")
    
    # USA
    elif any(x in i.lower() for x in ["tulsa","usa"]):
        country_list.append("USA")
    
    # Thailand
    elif any(x in i.lower() for x in ["chiang mai", "mae sot","mae sariang","mae hong son", "bangkok","thailand"]):
        country_list.append("Thailand")
    
    # India
    elif any(x in i.lower() for x in ["new delhi","mizoram","west delhi","manipur","india"]):
        country_list.append("India")
    
    # Austria   
    elif any(x in i.lower() for x in ["vienna","austria"]):
        country_list.append("Austria")
    
    # Germany
    elif any(x in i.lower() for x in ["essen","germany"]):
        country_list.append("Germany")
    
    # Czech Republic
    elif any(x in i.lower() for x in ["prague","czech republic"]):
        country_list.append("Czech Republic")

In [60]:
# Create City list
city_list =[]
# Ordered list in x of city names (Exhaustive and must be updated when website has more listings)
x = ["Yangon","Chin State","Moulmein","Ayeyarwady","Bogale","Mandalay","Tulsa","Chiang Mai", "Mae Sot","Mae Sariang","Mae Hong Son", 
     "Bangkok","New Delhi","Mizoram","West Delhi","Manipur","Vienna","Essen","Prague"]

#for i in address_list:

for i in range(len(address_list)):
    # City names are searched in order of list in x
    for a in x:
        # Append city names if found
        if a.lower() in address_list[i].lower():
            city_list.append(a)
            break
        # if no city names found in address, use country instead
        if a == x[-1] and (a.lower() in address_list[i].lower()) != 1:
            city_list.append(country_list[i])


In [52]:
import pandas as pd

# all original variables
df0 = pd.DataFrame({"Organisation":org_list,
                   "NGO Listing Page":listing_url_list,
                   "Classified Category":category_list,
                   "Address":address_list,
                   "Telephone":telephone_list,
                   "Mobile":mobile_list,
                   "Email":email_list,
                   "Website":website_list,
                   "Twitter":twitter_list,
                   "Facebook":facebook_list,
                   "Description":description_list,
                   "Name in Myanmese":local_name_list,
                   "Activities":activities_list,
                   "Registration ID":regid_list,
                   "Year of Foundation":yof_list,
                   "Further Links": further_web_list
                  })

# Processed mined data for EA
df = pd.DataFrame({"organisation":org_list,
                   "description":description_list,
                   "website":website_list,
                   "cause_area":category_list,
                   "programme_types":activities_list,
                   "address":address_list,
                   "country":country_list,
                   "city":city_list,
                   "contact_number":contact_list,
                   "email":email_list
                  })


In [53]:
df0 = df0[["Organisation","NGO Listing Page","Classified Category",
         "Registration ID","Year of Foundation","Address",
         "Telephone","Mobile","Email",
         "Website","Twitter","Facebook",
         "Description","Activities","Name in Myanmese","Further Links"]]

df = df[["organisation","description","website","cause_area","programme_types","address","contact_number","email"]]

In [54]:
df.head(20)

,organisation,description,website,cause_area,programme_types,address,contact_number,email
0,Maggin Development Consultancy Group,\nMDCG was founded by a group of 13 like-minde...,http://www.mdcg-myanmar.org,"Emergency Relief/Water, Sanitation and Hygiene...",\nActivities\n\nEmergency Respond\nWater and S...,"Yadanar Myaing Street, Kamayut Township, Yango...","Tel: +95 0949641582, Mob: +95 73058970",mdcg@mdcg-myanmar.org
1,Youth Pacemaker Organization,\nParticipation in Good Governance\n,NA,Capacity Building,\nActivities\nCapacity Building\nParticipation...,"Mandalay Region, Myanmar (Burma)","Tel: NA, Mob: +95 9798933530",youthpacemaker@gmail.com
2,Helping Hands Myanmar,\nHelping Hands Organization has experiences o...,NA,Capacity Building/Advocacy/Film/Livelihood and...,"\nActivities\nTraining,\nWorkshop,\nConference...","Helping Hands office, Kalay, Chin State, Myanm...","Tel: +95 9262281828, Mob: +95 9 967196555",hho.information@gmail.com
3,Stiftunglife,\nDie Stiftunglife ist eine 1989 gegründete de...,http://www.Stiftunglife.de,School/Capacity Building/Vocational Training/P...,\nActivities\nIm Sommer 2016 werden die Swimmi...,"Yangon, Yangon Region, Myanmar (Burma)","Tel: 00495141207173, Mob: 00491704512312",info@stiftunglife.de
4,Yello generation wave,\nWe are providing children all over the count...,NA,Children,\nActivities\nwe have nearly 200 children in m...,89MILE 5FALLON YANGON MANDALAY HIGHWAY ROAD,"Tel: 09780000085, Mob: 09785757620",NA
5,Brainworks-Total International School Yangon,\nBrainworks-Total Group of schools are locate...,http://www.brainworks-total.com,Education,\nActivities\nBrainworks-Total Group of school...,"41, Myanmar (Burma)","Tel: NA, Mob: NA",NA
6,Australia-Asia Program to Combat Trafficking i...,\nThe Australia-Asia Program to Combat Traffic...,http://www.aaptip.org/,Legal Support,\nActivities\n\n12th Bilateral Meeting between...,"No. 100, Kan Kaw Myaing Road, Parami, Yankin T...","Tel: +95-1657975, Mob: NA",info@aaptip.org
7,Agency for Education of the Poor Rural Youth,\nAgency for Education of the Poor Rural Youth...,http://www.aeprymm.com,"Rural Development/Vocational Training/Water, S...",\nActivities\nAEPRY has implemented five seven...,"Yangon, Yangon Region, Myanmar (Burma)","Tel: +95 - 931449092, Mob: +95 - 9261644801",info@aeprymm.com
8,"Zomi Innkuan USA, Inc.",\nZomi Innkuan USA is incorporated in the Stat...,http://www.zomicommunityusa.org/,Social Service/Advocacy/Human Rights/Women/Hum...,\nActivities\nRepresentative of Zomi Innkuan U...,"7438 S. 95th East Ave. Tulsa, OK 74133, USA","Tel: +1-918-576-4174, Mob: NA",zomicommunityusa@gmail.com
9,International Schools in Yangon,\nProviding best educational services in Yango...,http://www.brainworks-total.com,Education,\nActivities\nBrainworks-Total best educationa...,"Brainworks-Total K-12 Integrated School, 1 Thu...","Tel: 8551361, Mob: 9518551360",brainwork.total@gmail.com


In [61]:
# Some data cleaning
# Description: Remove all "\n" characters
df['description']=df['description'].str.replace("\n","")
# Activities: Remove all entries beginning with "\nActivities\n"
df['programme_types']=df['programme_types'].str.replace("\nActivities\n","")
df['programme_types']=df['programme_types'].str.replace("\n\n"," ")
df['programme_types']=df['programme_types'].str.replace("\n"," ")


# Some data cleaning
# Description: Remove all "\n" characters
df0['Description']=df0['Description'].str.replace("\n","")
# Activities: Remove all entries beginning with "\nActivities\n"
df0['Activities']=df0['Activities'].str.replace("\nActivities\n","")
df0['Activities']=df0['Activities'].str.replace("\n\n","")
df0['Activities']=df0['Activities'].str.replace("\n","")


In [62]:
df.head(20)

,organisation,description,website,cause_area,programme_types,address,contact_number,email
0,Maggin Development Consultancy Group,MDCG was founded by a group of 13 like-minded ...,http://www.mdcg-myanmar.org,"Emergency Relief/Water, Sanitation and Hygiene...",Emergency Respond Water and Sanitation Basic...,"Yadanar Myaing Street, Kamayut Township, Yango...","Tel: +95 0949641582, Mob: +95 73058970",mdcg@mdcg-myanmar.org
1,Youth Pacemaker Organization,Participation in Good Governance,NA,Capacity Building,Capacity Building Participation in Governance ...,"Mandalay Region, Myanmar (Burma)","Tel: NA, Mob: +95 9798933530",youthpacemaker@gmail.com
2,Helping Hands Myanmar,Helping Hands Organization has experiences of ...,NA,Capacity Building/Advocacy/Film/Livelihood and...,"Training, Workshop, Conference","Helping Hands office, Kalay, Chin State, Myanm...","Tel: +95 9262281828, Mob: +95 9 967196555",hho.information@gmail.com
3,Stiftunglife,Die Stiftunglife ist eine 1989 gegründete deut...,http://www.Stiftunglife.de,School/Capacity Building/Vocational Training/P...,Im Sommer 2016 werden die Swimming Doctors III...,"Yangon, Yangon Region, Myanmar (Burma)","Tel: 00495141207173, Mob: 00491704512312",info@stiftunglife.de
4,Yello generation wave,We are providing children all over the country...,NA,Children,we have nearly 200 children in my monastry.,89MILE 5FALLON YANGON MANDALAY HIGHWAY ROAD,"Tel: 09780000085, Mob: 09785757620",NA
5,Brainworks-Total International School Yangon,Brainworks-Total Group of schools are located ...,http://www.brainworks-total.com,Education,Brainworks-Total Group of schools are located ...,"41, Myanmar (Burma)","Tel: NA, Mob: NA",NA
6,Australia-Asia Program to Combat Trafficking i...,The Australia-Asia Program to Combat Trafficki...,http://www.aaptip.org/,Legal Support,12th Bilateral Meeting between Myanmar Police...,"No. 100, Kan Kaw Myaing Road, Parami, Yankin T...","Tel: +95-1657975, Mob: NA",info@aaptip.org
7,Agency for Education of the Poor Rural Youth,Agency for Education of the Poor Rural Youth (...,http://www.aeprymm.com,"Rural Development/Vocational Training/Water, S...",AEPRY has implemented five seven rural develop...,"Yangon, Yangon Region, Myanmar (Burma)","Tel: +95 - 931449092, Mob: +95 - 9261644801",info@aeprymm.com
8,"Zomi Innkuan USA, Inc.",Zomi Innkuan USA is incorporated in the State ...,http://www.zomicommunityusa.org/,Social Service/Advocacy/Human Rights/Women/Hum...,Representative of Zomi Innkuan USA which is in...,"7438 S. 95th East Ave. Tulsa, OK 74133, USA","Tel: +1-918-576-4174, Mob: NA",zomicommunityusa@gmail.com
9,International Schools in Yangon,Providing best educational services in Yangon ...,http://www.brainworks-total.com,Education,Brainworks-Total best educational school in Ya...,"Brainworks-Total K-12 Integrated School, 1 Thu...","Tel: 8551361, Mob: 9518551360",brainwork.total@gmail.com


In [59]:
# Output to csv
df0.to_csv("myanmar_ngo_all_data.csv")
df.to_csv("myanmar_ngo_EA_data.csv")